In [1]:
import igraph as ig
import utils
import numpy as np
import itertools
from statistics import mean


In [2]:
def VISUAL_STYLE(g: ig.Graph) -> dict:
    return {
        'layout': utils.layout_geo(g),
        'bbox': (0, 0, 3000, 3000)
    }


In [3]:
g = ig.Graph.Read('../../data/graphs/april2022.graphml')


In [4]:
g_L_space = ig.Graph(directed=True)


In [5]:
# Adding all vertex from original graph
g_L_space.add_vertices(
    n=g.vs['name'],
    attributes={
        'label': g.vs['label'],
        'lat': g.vs['lat'],
        'lon': g.vs['lon']
    }
)


In [6]:
for a, b in itertools.permutations(g.vs, 2):
    edges = g.es.select(_source=a.index, _target=b.index)
    if len(edges) > 0:
        g_L_space.add_edge(
            source=a.index,
            target=b.index,
            time_second=mean([e['time_second'] for e in edges]),
            num_train=mean([e['num_train'] for e in edges])
        )


In [7]:
g_L_space.summary()


'IGRAPH DN-- 428 1173 -- \n+ attr: label (v), lat (v), lon (v), name (v), num_train (e), time_second (e)'

In [8]:
betweeness = np.array(g_L_space.betweenness(directed=True))
norm_btwness = (betweeness - betweeness.min()) / (betweeness.max() - betweeness.min()) 

vs = {
    **VISUAL_STYLE(g_L_space),
    'vertex_size': 50*norm_btwness
}

print(f'Top 10 vertex by betweenness (normalized):')
for i in utils.top_n_indices(norm_btwness, 10):
    print(f"label: {g.vs[i]['label']}, betweeness: {norm_btwness[i]}")

ig.plot(g_L_space, **vs);

Top 10 vertex by betweenness (normalized):
label: MILANO LAMBRATE, betweeness: 1.0
label: MILANO ROGOREDO, betweeness: 0.7863961292274942
label: MONZA, betweeness: 0.77003784379373
label: MILANO CENTRALE, betweeness: 0.6256446891017553
label: TREVIGLIO, betweeness: 0.5527981125491678
label: LECCO, betweeness: 0.5071597956744601
label: MILANO PORTA GARIBALDI, betweeness: 0.475885434347367
label: RHO FIERA, betweeness: 0.4434781021500448
label: MILANO BOVISA FNM, betweeness: 0.44309634516137064
label: BRESCIA, betweeness: 0.426197251601166


In [9]:
g_L_space.vs['total_train'] = [
    max(
        sum(edge['num_train'] for edge in g_L_space.es.select(_source=vertex.index)),
        sum(edge['num_train'] for edge in g_L_space.es.select(_target=vertex.index))
    )
    for vertex in g_L_space.vs
]

In [11]:
print('Assortativity by vertex degree:', g.assortativity_degree())
print('L-space assortativity by vertex degree:', g_L_space.assortativity_degree())
print('L-space assortativity by vertex total trains, directed:', g_L_space.assortativity('total_train'))

Assortativity by vertex degree: 0.5288859346490613
L-space assortativity by vertex degree: 0.2282961704369721
L-space assortativity by vertex total trains, directed: 0.5158865851932436
